# LELA32051 Computational Linguistics Week 8

This week we are going first to take a look at the challenge of machine translation.

We'll look at German-to-English MT. Here is a set of sentences - the s stands for source and the t for target. Hopefully the translations here will be somewhat transparent to you. The only thing that might not be obvious is the use of "ja". This means "yes" in some context but is also use to mean something like "certainly". So "das haus ist ja gros" could be translated as "the house is certainly big" but because there isn't a perfect match from ja to certainly it tends to just be omitted in English translation as it is here.

In [ ]:
s1='klein ist das haus '
t1='the house is small '
s2='das haus ist ja groß '
t2='the house is big '
s3='ja das buch ist klein '
t3='yes the book is small '
s4='das haus '
t4='the house '
s5='ein buch '
t5='a book '

We are going to use the now very familiar re.sub function to perform translation first. 
The g2e function takes German as input and should output English.

Its translation is performed using a series of re.sub functions.

First let's take a really naive approach.

In [ ]:
import re

In [ ]:
def g2e(out):
    re.UNICODE
    out=re.sub("klein ","small ",out)
    out=re.sub("ist ","is ",out)
    out=re.sub("das ","the ",out)
    out=re.sub("haus ","house ",out)
    out=re.sub("groß ","big ",out)
    out=re.sub("buch ","book ",out)
    out=re.sub("ein ","a ",out)
    out=re.sub("ja ","yes ",out)
    
    return out


In [ ]:
print(g2e(s1) + "\n" + g2e(s2)  + "\n" + g2e(s3)  + "\n" + g2e(s4)  + "\n" + g2e(s5))


That didn't work well. Your job is to change the rules so that the function returns the correct translation.

To make your job easier I have marked the part of speech using the following tags, based on what an automatic part of speech tagger would do (we'll look at these and how they work next week).

ADJ : adjective
AUX : auxiliary verb
ART : article/determiner
N : noun
ADV : adverb

You can make use of the tags by matching them and their associated words like this:

[^ ]+_ART

so if you wrote

re.sub("([^ ]+)_ART",\\1,out)

then it would return an article without its tag.

In [ ]:
s1='klein_ADJ ist_AUX das_ART haus_N'
t1='the house is small'
s2='das_ART haus_N ist_AUX ja_ADV groß_ADJ '
t2='the house is big '
s3='ja_ADV das_ART buch_N ist_AUX klein_ADJ'
t3='yes the book is small '
s4='das_ART haus_N'
t4='the house '
s5='ein_ART buch_N'
t5='a book '

In [ ]:
def g2e(out):
    re.UNICODE
    out=re.sub('klein_','small_',out)
    out=re.sub('ist_','is_',out)
    out=re.sub('das_','the_',out)
    out=re.sub('haus_','house_',out)
    out=re.sub('ja_','yes_',out)
    out=re.sub('groß_','big_',out)
    out=re.sub('buch_','book_',out)
    out=re.sub('ein_','a_',out)
    
    out = re.sub("_[^ ]+","",out)
    return out


In [ ]:
print(g2e(s1) + "\n" + g2e(s2)  + "\n" + g2e(s3)  + "\n" + g2e(s4)  + "\n" + g2e(s5))

### Another sentence set to explore 

Update the below function to translate these sentence pairs in as few a set of rules as possible

In [ ]:
s1="der_ART mann_N hat_AUX fußball_N gespielt_V"
t1="the man played football"
s2="der_ART mann_N spielt_V fußball_N" 
t2="the man plays football"
s3="der_ART mann_N hat_AUX kartoffeln_N gekocht_V"
t3="the man cooked potatoes"
s4="der_ART mann_N kocht_V kartoffeln_N"
t4="the man cooks potatoes"

In [ ]:
def g2e(out):
    re.UNICODE
    out=re.sub('der_','the_',out)
    out=re.sub('mann_','man_',out)
    out=re.sub('fußball_','football_',out)
    out=re.sub('spielt_','plays_',out)
    out=re.sub('kocht_','cooks_',out)
    out=re.sub('kartoffeln_','potatoes_',out)
    
    out = re.sub("_[^ ]+","",out)
    return out

In [ ]:
print(g2e(s1) + "\n" + g2e(s2)  + "\n" + g2e(s3)  + "\n" + g2e(s4)  + "\n")

And if you are really feeling brave, try accounting for these too:

In [ ]:
s5="der_ART mann_N spielt_V gerne_ADV fußball_N" 
t5="the man likes playing football"
s6="der_ART mann_N hat_AUX gerne_ADV fußball_N gespielt_V" 
t6="the man liked to play football"

## Statistical machine translation

We will look next at statistical machine translation. NLTK has some built in tools for this that we can make use of.

To make sure we have latest version of nltk let's install and then restart runtime.

In [ ]:
!pip install --user -U nltk

In [ ]:
import nltk
nltk.download('punkt')
import math
from nltk import AlignedSent
from nltk import IBMModel3

### Build a translation table

We start by performing alignment and building a translation table

In [ ]:
s1='klein ist das haus'
t1='the house is small'
s2='das haus ist ja groß'
t2='the house is big'
s3='das buch ist klein'
t3='the book is small'
s4='das buch'
t4='the book'
s4='das house'
t4='the house'

In [ ]:
parallel_corpus = []
parallel_corpus.append(AlignedSent(nltk.word_tokenize(s1),nltk.word_tokenize(t1)))
parallel_corpus.append(AlignedSent(nltk.word_tokenize(s2),nltk.word_tokenize(t2)))
parallel_corpus.append(AlignedSent(nltk.word_tokenize(s3),nltk.word_tokenize(t3)))
parallel_corpus.append(AlignedSent(nltk.word_tokenize(s4),nltk.word_tokenize(t4)))

In [ ]:
ibm3 = IBMModel3(parallel_corpus, 50)

In [ ]:
ibm3.translation_table['haus']

You can download and train on a larger aligned corpus by running this code (but beware it will take quite a while):

import nltk <br>
nltk.download('comtrans') <br>
ende=comtrans.aligned_sents('alignment-de-en.txt') <br>
ende_subset = ende[1:100] <br>
ibm3 = IBMModel3(ende_subset, 2) <br>

In [ ]:
phrase_table = nltk.translate.PhraseTable()
for triple in ibm3.translation_table.items():
      for i in triple[1].items():
            phrase_table.add((triple[0],),(i[0],),math.log(i[1]))
    

In [ ]:
phrase_table.translations_for(('ist',))

### Build a probabilistic language model

We will use the collected works of Jane Austen here, but in real systems you would want to use a larger and more representative corpus.

In [ ]:
!wget https://www.gutenberg.org/files/31100/31100.txt    
f = open('31100.txt',"r",encoding='windows-1252')
text = f.read()  
text = text + "\n" + t1 + "\n" + t2 + "\n" + t3 + "\n" + t4 + "\n" + t5 + "\n"
tokenized_text = [list(map(str.lower, nltk.word_tokenize(sent))) 
                  for sent in nltk.sent_tokenize(text)]

In [ ]:
import nltk.lm.preprocessing
n = 3
train_data, padded_sents = nltk.lm.preprocessing.padded_everygram_pipeline(n, tokenized_text)

In [ ]:
from nltk.lm import MLE
model = MLE(n) 

In [ ]:
model.fit(train_data, padded_sents)


In [ ]:
model.generate(8)

In [ ]:
from collections import defaultdict
language_prob = defaultdict(lambda: -99.0)
for t in nltk.ngrams(nltk.word_tokenize(t1 + " " + t2 + " " + t3 + " " + t4),3):
    if model.logscore(t[2],[t[0],t[1]]) < 0:
        language_prob[t] = model.logscore(t[2],[t[0],t[1]])
    else:
        language_prob[t] = -999
language_model = type('',(object,),{'probability_change': lambda self, context, phrase: language_prob[phrase], 'probability': lambda self, phrase: language_prob[phrase]})()

In [ ]:
language_prob.items()

### Combine with translation model to perform decoding


In [ ]:
stack_decoder = nltk.translate.StackDecoder(phrase_table, language_model)

In [ ]:
stack_decoder.distortion_factor = 1
stack_decoder.word_penalty = 0

In [ ]:
stack_decoder.translate(nltk.word_tokenize("das haus ist groß"))

In [ ]:
stack_decoder.translate(nltk.word_tokenize("klein ist das haus"))